In [ ]:
!pip show spacy  

Name: spacy
Version: 3.4.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: jinja2, typing-extensions, setuptools, packaging, murmurhash, wasabi, tqdm, numpy, cymem, langcodes, srsly, spacy-loggers, spacy-legacy, pydantic, catalogue, pathy, typer, thinc, preshed, requests
Required-by: fastai, en-core-web-sm


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim
import spacy
import pandas as pd
import numpy as np
from gensim.utils import tokenize
from sklearn.metrics import f1_score
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc


In [ ]:
def avg_sentence_len(text):
  sentences = text.split(".") #split the text into a list of sentences.
  words = text.split(" ") #split the input text into a list of separate words
  if(sentences[len(sentences)-1]==""): #if the last value in sentences is an empty string
    average_sentence_length = len(words) / len(sentences)-1
  else:
    average_sentence_length = len(words) / len(sentences)
  return average_sentence_length


def Average(lst):
    return sum(lst) / len(lst)  

def isWordPresent(sentence, word):
 
    # To convert the word in uppercase
    word = word.upper()
 
    # To convert the complete
    # sentence in uppercase
    sentence = sentence.upper()
 
    # splitting the sentence to list
    lis = sentence.split()
    # checking if word is present
    if(lis.count(word) > 0):
        return True
    else:
        return False    

# Training Data -  To train the model, we’ll need some training data
NER annotation this part is to recognize the entity by labeling various entities like name, location, month, year, day and organization. 

In [ ]:
training_data = {"classes":["PERSON","ORG","DAY","LOCATION","YEAR","GPE","MONTH"],"annotations":[["Ang Gahum sa Budget sa Pangulo naa sa Seksyon 22, Artikulo 7 sa Konstitusyon nga naglatid nga ang Presidente magpakita sa Kongreso katloan ka adlaw gikan sa matag pag-abli sa regular nga sesyon (ika-4 nga Lunes sa Hulyo) sa usa ka National Expenditure Program (NEP).\r",{"entities":[[23,30,"PERSON"],[98,108,"PERSON"],[122,130,"ORG"],[195,200,"DAY"],[205,210,"DAY"],[214,219,"MONTH"],[231,265,"ORG"]]}],["Ang NEP o ang gisugyot nga badyet alang sa sunod tuig mao ang basihan aron makapasar ang Kongreso sa usa ka General Appropriation Bill (GAB).\r",{"entities":[[4,7,"ORG"],[89,97,"ORG"]]}],["Ang tinuig nga NEP o gisugyot nga badyet gihimo sa Department of Budget and Management (DBM) nga naglatid sa mga gasto o badyet sa tanan nga ahensya sa gobyerno sa sunod tuig.\r",{"entities":[[15,18,"ORG"],[51,92,"ORG"],[152,160,"ORG"]]}],["Alang sa umaabot nga 2021, ang DBM nagsumite sa Kamara de Representante sa NEP o gisugyot nga nasudnon nga badyet sa kantidad nga P4.506 trilyon nga dili madugangan o madugangan sa Kongreso tungod kay gidili sa Artikulo 6, Seksyon 25 (No. 1) sa Batakang Balaod.\r",{"entities":[[21,25,"YEAR"],[31,34,"ORG"],[75,78,"ORG"],[181,189,"ORG"]]}],["Ang Konstitusyon naghatag sa Presidente nga Gahum sa Budget sa Pangulo tungod kay labi nga naa sa posisyon nga mahibal-an kung unsa ang kinahanglan sa lainlaing mga ahensya sa gobyerno alang sa maayong operasyon niini.\r",{"entities":[[4,16,"ORG"],[29,39,"PERSON"],[63,70,"PERSON"],[176,184,"ORG"]]}],["Ang Gahum sa Purse o gahum sa angay nga pondo sa gobyerno alang sa mga gasto ug operasyon niini naa sa kamot sa Kongreso, partikular ang House of Representatives (House).\r",{"entities":[[49,57,"ORG"],[112,120,"ORG"],[137,170,"ORG"]]}],["Ang Kongreso adunay gahum sa pagbag-o sa gisugyot nga mga badyet nga gilatid sa NEP.\r",{"entities":[[4,12,"ORG"],[80,83,"ORG"]]}],["Mao nga mahimo sa Kongreso aron madugangan o maibanan ang badyet nga girekomenda sa Presidente o DBM alang sa usa ka proyekto o ahensya sa gobyerno.\r",{"entities":[[18,26,"ORG"],[84,94,"PERSON"],[97,100,"ORG"],[139,147,"ORG"]]}],["Ang bugtong nga gidili sa Konstitusyon mao ang pagdugang sa kinatibuk-ang badyet nga girekomenda sa Presidente sa Kongreso.\r",{"entities":[[100,110,"PERSON"],[114,122,"ORG"]]}],["Tungod sa gahum sa pitaka sa Kongreso, tanan nga mga proyekto sa Presidente ug ehekutibo nga departamento mahimo nga dili magpadayon kung dili kini gipondohan sa Kongreso.\r",{"entities":[[29,37,"ORG"],[65,75,"PERSON"],[162,170,"ORG"]]}],["Bisan kung gusto sa Pangulo nga pondohan ang usa ka ahensya sa gobyerno o pondohan ang usa ka proyekto, wala kini gahum nga maghatag isyu salapi sa National Treasury tungod kay kini nga gahum talagsaon sa Kongreso.\r",{"entities":[[20,27,"PERSON"],[63,71,"ORG"],[148,165,"ORG"],[205,213,"ORG"]]}],["Si Las Piñas City Representative Camille Villar nagdumili sa nominasyon aron mahimong deputy speaker sa House of Representatives nunot sa laing pag-shuffle sa Kamara kaniadtong Miyerkules.\r",{"entities":[[3,17,"LOCATION"],[33,47,"PERSON"],[104,128,"ORG"],[177,187,"DAY"]]}],["Sa sulat nga gipadala niya kay Speaker Lord Allan Velasco, gipadayag ni Villar ang iyang pasalamat sa nominasyon nga himuon siya nga usa sa mga deputy speaker sa pamunoan\r",{"entities":[[39,57,"PERSON"],[72,78,"PERSON"]]}],["Bisan pa, gipanghimatuud sa mas bata nga si Villar si Velasco nga hingpit niya nga gisuportahan ang agenda sa pamalaod sa bag-ong pamunuan sa Kamara.\r",{"entities":[[44,50,"PERSON"],[54,61,"PERSON"],[142,148,"ORG"]]}],["Lakip na si Rep. Si Villar lakip sa tulo nga kongresista nga gitudlo sa plenary kaniadtong Miyerkules aron mahimong deputy speaker - kauban ang representante sa Cagayan De Oro City nga si Rufus Rodriguez ug Buhay party-list Representative Lito Atienza.\r",{"entities":[[20,26,"PERSON"],[45,56,"PERSON"],[91,101,"LOCATION"],[161,180,"LOCATION"],[188,203,"PERSON"],[239,251,"PERSON"]]}],["Gihulipan ni Capiz Rep. Fredenil Castro, Laguna Rep. Dan Fernandez, ug Batangas Rep. Raneo Abu, nga naila nga suod nga kaalyado ni kanhi Speaker Alan Peter Cayetano .\r",{"entities":[[13,39,"PERSON"],[41,47,"LOCATION"],[48,66,"PERSON"],[71,79,"LOCATION"],[80,94,"PERSON"],[145,164,"PERSON"]]}],["Lima nga menor de edad nga gipasanginlang molestiya sa online ang naluwas sa pulisya sa Pasig City, pinauyon sa report ni James Agustin sa Unang Balita kaniadtong Huwebes.\r",{"entities":[[88,98,"LOCATION"],[122,135,"PERSON"],[139,151,"ORG"],[163,170,"DAY"]]}],["Ang bugaw, kinsa menor de edad usab, gidakop human sa gihimong entrapment operation sa mga awtoridad sa Barangay Pinagbuhatan kaniadtong Miyerkules sa gabii.\r",{"entities":[[104,112,"ORG"],[137,147,"DAY"]]}],["Tulo sa mga tigluwas 16 anyos samtang ang laing duha 15 anyos.\r",{"entities":[]}],["Pinauyon sa otoridad, ang mga biktima nagbugaw sa kantidad nga P3,000.\r",{"entities":[]}],["Ang mga biktima itugyan sa Department of Social Welfare and Development samtang ang bugaw mapasakaan og kasong kalapasan sa Expanding Anti-Trafficking of Persons Act.\r",{"entities":[[27,71,"ORG"]]}],["Nagpatuman usa ka bag-ong shuffle sa House of Representatives.\r",{"entities":[[37,61,"ORG"]]}],["Lakip sa gitangtang sa katungdanan ang pipila ka mga kaalyado ni kanhi Speaker Alan Peter Cayetano, kinsa gipulihan ni Speaker Lord Allan Velasco kaniadtong Oktubre.\r",{"entities":[[71,98,"PERSON"],[119,145,"PERSON"],[157,164,"MONTH"]]}],["Sa sesyon kaniadtong Miyerkules, ang Representante sa Capiz nga si Fredenil Castro, Laguna Rep. gitangtang isip representante sa tigpamaba Dan Fernandez, ug Batangas Rep. Raneo Abu .\r",{"entities":[[21,31,"DAY"],[67,82,"PERSON"],[84,90,"LOCATION"],[139,152,"PERSON"],[157,165,"LOCATION"],[166,180,"PERSON"]]}],["Gitudlo nga si Cagayan De Oro City Rep. Rufus Rodriguez, Life party-list Rep. Lito Atienza, ug Las Piñas City Rep. Camille Villar .\r",{"entities":[[15,34,"LOCATION"],[35,55,"PERSON"],[73,90,"PERSON"],[95,109,"LOCATION"],[110,129,"PERSON"]]}],["Si Camarines Sur Rep kaniadto gitangtang isip deputy speaker. Si Luis Raymund Villafuerte, naila usab nga kaalyado ni Cayetano .\r",{"entities":[[3,16,"LOCATION"],[65,89,"PERSON"],[118,126,"PERSON"]]}],["Samtang, si Pampanga Rep. gitudlo nga chairman sa House Committee on Energy. Mikey Arroyo.\r",{"entities":[[12,20,"LOCATION"],[77,89,"PERSON"]]}],["Ang mao nga komite gipangulohan kaniadto ni Velasco sa wala pa siya mapili nga pinuno sa Kamara.\r",{"entities":[[44,51,"PERSON"],[89,95,"ORG"]]}],["Si Caloocan City Rep. Dale Along Malapitan isip pinuno sa board sa House of Representative Electoral Tribunal.\r",{"entities":[[3,16,"LOCATION"],[17,42,"PERSON"],[67,109,"ORG"]]}],["Si ACT-CIS party-list Rep nga gihimong vice chairman sa House Committee on Account. Eric Go Yap, pinangunahan ni Davao City Rep. Paolo Duterte .\r",{"entities":[[3,21,"ORG"],[84,95,"PERSON"],[113,123,"LOCATION"],[124,142,"PERSON"]]}],["Si Velasco gitudlo nga tigbantay sa lehislatibo sa unang distrito sa Dakbayan sa Cebu pagkahuman sa pagpanaw ni Rep. Raul Del Mar .\r",{"entities":[[3,10,"PERSON"],[81,85,"LOCATION"],[112,129,"PERSON"]]}],["Pinauyon kay Deputy Majority Leader Xavier Jesus Romualdo, kini ang hangyo sa pamilya ni Del Mar.\r",{"entities":[[36,57,"PERSON"],[89,96,"PERSON"]]}],["Si kanhi Batangas Rep. magsilbi nga bag-ong Kalihim sa Heneral sa Kamara. Mark Llandro Mendoza, pagkahuman ni Atty. Jocelia Bighani Sipin .\r",{"entities":[[9,17,"LOCATION"],[66,73,"ORG"],[74,94,"PERSON"],[110,137,"PERSON"]]}],["Kaniadtong Oktubre, giingon ni Cayetano nga kinahanglan nga wala’y shuffle ubos sa bag-ong pamunuan sa Kamara.\r",{"entities":[[11,18,"MONTH"],[31,39,"PERSON"],[103,109,"ORG"]]}],["Gipahayag ni Defense Secretary Delfin Lorenzana nga si Bise Presidente Leni Robredo wala mosakay sa C-130 military aircraft sa pag-adto niya sa Bicol aron ihatud ang mga hinabang sa mga biktima sa bagyo.\r",{"entities":[[31,47,"PERSON"],[55,83,"PERSON"],[144,149,"LOCATION"]]}],["Sa baylo, nagpakita nga ang relief goods lamang gikan sa opisina ni Robredo ang gisakay sa helikopter sa Air Force kaniadtong Nobyembre 3 hangtod sa Catanduanes.\r",{"entities":[[68,75,"PERSON"],[126,135,"MONTH"],[136,137,"DAY"],[149,160,"LOCATION"]]}],["Ang post sa reporter sa GMA News nga si Joseph Morong sa Twitter nagpakita sa kopya sa flight manifest.\r",{"entities":[[24,32,"ORG"],[40,53,"PERSON"],[57,64,"ORG"]]}],["Tungod niini, nangayo og pasaylo si Lorenzana kay Robredo.\r",{"entities":[[36,45,"PERSON"],[50,57,"PERSON"]]}],["Sa sayo pa, mireklamo usab si Chief Presidential Legal Counsel Salvador Panelo nga si Robredo nagsakay sa usa ka C-130 nga eroplano, nga gilimod gilayon sa opisina sa bise presidente.\r",{"entities":[[30,78,"PERSON"],[86,93,"PERSON"],[167,182,"PERSON"]]}],["Sa kasubo, gitabangan ni Bise Presidente Leni ang mga naapektuhan sa mga bagyo ug baha imbis nga suportahan ang bato, pan-vandalismo ug mga bakak nga bato, dugang pa niya.\r",{"entities":[[25,45,"PERSON"]]}],["Ang tigpamaba sa Armed Forces nga si Major General Edgard Arevalo nagsulti nga wala’y problema bisan kung ang bise presidente mogamit kagamitan sa militar aron matabangan ang mga biktima sa kalamidad.\r",{"entities":[[17,29,"ORG"],[37,65,"PERSON"],[110,125,"PERSON"],[147,154,"ORG"]]}],["usa ka toro sa usa ka tindahan sa Tsina\r",{"entities":[[34,39,"GPE"]]}],["Miadto ako sa Amerika aron magtuon.\r",{"entities":[[14,21,"GPE"]]}],["Gusto kong moadto sa Thailand sa pila ka adlaw.\r",{"entities":[[21,29,"GPE"]]}],["Gisugyot nako nga sulayan niya ang pag adto sa Singapore.\r",{"entities":[[47,56,"GPE"]]}],["Gisultihan kami sa magtutudlo nga nadiskubrehan ni Columbus sa Pransiya kaniadtong 1492.\r",{"entities":[[63,71,"GPE"]]}],["Ang akong anak nga lalaki moadto sa Vietnam aron magtuon sa medisina.\r",{"entities":[[36,43,"GPE"]]}],["Tungod kay nasamdan siya nakadesisyon siya nga mobalik sa Germany.\r",{"entities":[[58,65,"GPE"]]}],["Gisugdan niya kami pagsulti sa iyang mga kasinatian sa iyang pag estar sa Afghanistan. Kitang tanan mga dalunggan.\r",{"entities":[[74,86,"GPE"]]}],["Gitudloan niya kami nga nadiskubrehan ni Columbus ang Australia.\r",{"entities":[[54,63,"GPE"]]}],["Miadto siya sa Argentina alang sa katuyoan nga mahibal - an ang usa ka bag - ong pamaagi sa paghimo og mga tanum.\r",{"entities":[[15,24,"GPE"]]}],["Ang Papahānaumokuākea Marine National Monument mas dako kaysa sa tanan nga nasyonal nga mga parke sa Belgium!\r",{"entities":[[101,108,"GPE"]]}],["Ang kinatas - an nga 25 managsama nga mga managers sa pondo sa hedge naghimo labaw pa sa tanan nga mga magtutudlo sa kindergarten sa Brazil.\r",{"entities":[[133,139,"GPE"]]}],["Sa Tsina nagpanuko kami sa pag - ila nga ang mga sumbanan sa sinultihan gapos sa sosyal nga klase.\r",{"entities":[[3,8,"GPE"]]}],["Ang una nga mga makina nga nagpatik nag - abut sa Estados Unidos kaniadtong tungatunga sa 1600, nga nagtimaan sa usa ka hinungdanon nga lakang sa kasaysayan sa libre nga press sa Amerika.\r",{"entities":[[50,64,"GPE"],[179,186,"GPE"]]}],["Miadto siya sa Cambodia alang sa katuyoan nga mahibal - an ang usa ka bag - ong pamaagi sa paghimo og mga tanum.\r",{"entities":[[15,23,"GPE"]]}],["Miadto siya sa Canada alang sa katuyoan nga magtuon sa panitik sa Canada.\r",{"entities":[[15,21,"GPE"],[66,72,"GPE"]]}],["Gisugdan niya kami pagsulti sa iyang mga kasinatian sa iyang pag - estar sa Egypt. Kitang tanan mga dalunggan.\r",{"entities":[[76,82,"GPE"]]}],["Naugmad niya ang iyang kahanas sa Ingles samtang siya naa sa England.\r",{"entities":[[61,68,"GPE"]]}],["Napulo ka tuig ang nilabay gikan sa iyang pag - adto sa Greece.\r",{"entities":[[56,62,"GPE"]]}],["Pagkahuman sa usa ka miting sa Amerika nga nakahukom siya nga magsulat usa ka libro alang sa mga dili siyentista.\r",{"entities":[[31,38,"GPE"]]}],["Kung pangutan-on mo kami kung ang Bise Presidente naggamit naval, air, land assets, wala kami problema didto.\r",{"entities":[[34,49,"PERSON"]]}],["Mahimo naton siya buligan, gipatin-aw ni Arevalo sa virtual press conference .\r",{"entities":[[41,48,"PERSON"]]}],["Syempre, kinahanglan syempre, naa tay mga inisyal nga koordinasyon nga kinahanglan buhaton aron mahibal-an usab nila ang padulnganan nga adtoan, unsa ang security package nga kinahanglan naton ihatag sa kaniya, dugang pa niya .\r",{"entities":[]}],["Giklaro usab ni Arevalo nga wala nakadawat bisan unsang hangyo ang General General headquarters sa Robredo kalabot sa pagluwas sa mga na-trap sa baha.\r",{"entities":[[16,23,"PERSON"],[67,95,"ORG"],[99,106,"PERSON"]]}],["Ang mga senador nagkahiusa nga dili nila putlon ang pondo nga gigahin sa mga state college ug unibersidad (SUCs) alang sa 2021, suma ni Senate President Pro Tempore Ralph Recto.\r",{"entities":[[8,15,"PERSON"],[77,105,"ORG"],[107,111,"ORG"],[122,126,"YEAR"],[136,176,"PERSON"]]}],["Kaniadtong Martes sa gabii, nagbanta si Presidente Rodrigo Duterte nga mag-atras sa pondo gikan sa University of the Philippines (UP) alang sa giingon nga pagrekrut sa mga batan-on aron mahimong mga rebeldeng komunista.\r",{"entities":[[11,17,"DAY"],[40,66,"PERSON"],[99,128,"ORG"],[130,132,"ORG"],[172,180,"PERSON"],[199,218,"PERSON"]]}],["Gihimo ni Duterte ang pagsaway human ang mga estudyante sa Ateneo de Manila University (ADMU) nanawagan alang sa usa ka akademikong welga agig protesta sa giingon nga wala’y hinungdan nga tubag sa gobyerno sa mga kalamidad ug pandemics.\r",{"entities":[[10,17,"PERSON"],[45,55,"PERSON"],[59,86,"ORG"],[88,92,"ORG"],[197,205,"ORG"]]}],["Nanawagan si Senador Francis Kiko Pangilinan alang sa ligal nga pagtangtang sa mga pondo sa UP.\r",{"entities":[[13,44,"PERSON"],[92,94,"ORG"]]}],["Sa National Expenditure Program sa 2021 nga badyet, P20.8 bilyon ang gigahin alang sa kinatibuk-ang UP System.\r",{"entities":[[35,39,"YEAR"],[100,102,"ORG"]]}],["Pinauyon kay Recto, ang pasiunang pondo nga gi-aprobahan sa Department of Budget and Management alang sa mga SUC mahimong madugangan kung makita sa mga magbabalaod ang panginahanglan.\r",{"entities":[[13,18,"PERSON"],[60,95,"ORG"],[109,112,"ORG"]]}],["Gibungkag karon sa Senado ang sugyot sa badyet sa gobyerno alang sa 2021 nga nagkantidad og P4.5 trilyon.\r",{"entities":[[19,25,"ORG"],[50,58,"ORG"],[68,72,"YEAR"]]}],["Gitawag ni Presidente Rodrigo Duterte nga bakakon si Bise Presidente Leni Robredo tungod sa giingon nga pagpangita kaniya sa bagyong Ulysses.\r",{"entities":[[11,37,"PERSON"],[53,81,"PERSON"]]}],["Ang bise presidente adunay tubag sa iyang mga pangutana .\r",{"entities":[[4,19,"PERSON"]]}],["Sa pahayag sa telebisyon ni Duterte nga gipagawas kaniadtong Martes sa gabii, gipatin-aw sa presidente nga naa siya sa online nga ASEAN Summit sa pag-igo sa bagyo.\r",{"entities":[[28,35,"PERSON"],[61,67,"DAY"],[92,102,"PERSON"],[130,142,"ORG"]]}],["Kaniadtong Biyernes sa gabii diin gikataho nga nalunod ang Cagayan tungod sa pagbaha, si Robredo busy sa pagkontak sa militar alang sa pagluwas sa mga residente sa atop sa mga balay.\r",{"entities":[[11,19,"DAY"],[59,66,"LOCATION"],[89,96,"PERSON"],[118,125,"ORG"],[151,160,"PERSON"]]}],["Sukad gi-atake si Ulysses, wala’y pahayag sa publiko si Robredo bahin sa pagpangita sa presidente bisan kung ang Nasaan Ang Pangulo nauso sa social media .\r",{"entities":[[18,25,"PERSON"],[56,63,"PERSON"],[87,97,"PERSON"],[124,131,"PERSON"]]}],["Pinauyon kay Duterte, gisugdan ni Robredo ang Nasaan Ang Pangulo, nga nag-uso usab sa panahon sa bagyong Rolly.\r",{"entities":[[13,20,"PERSON"],[34,41,"PERSON"],[57,64,"PERSON"]]}],["Gipatin-aw sa pangulo, ginabantayan niya ang kahimtang sa epekto sa bagyo bisan kung nagtambong siya sa ASEAN Summit .\r",{"entities":[[14,21,"PERSON"],[104,116,"ORG"]]}],["Dili nimo sila kinahanglan orderon tungod kay duha ka adlaw sa wala pa, didto sila gipakatap, nagpadayon siya .\r",{"entities":[]}],["Pinauyon kay Duterte, naghatag kuno si Robredo sa mga militar sa kasagsagan sa bagyong Ulysses. .\r",{"entities":[[13,20,"PERSON"],[39,46,"PERSON"],[54,61,"ORG"]]}],["Sa mga tweet ni Robredo, gikonektar niya ang mga netizen nga nanginahanglan tabang sa mga rescue team sa lugar\r",{"entities":[[16,23,"PERSON"],[49,56,"PERSON"]]}],["Luya ka.\r",{"entities":[]}],["Gipasidan-an usab ni Duterte si Robredo kung ang ulahi modagan pagka-presidente sa piliay nga 2022.\r",{"entities":[[21,28,"PERSON"],[32,39,"PERSON"],[94,98,"YEAR"]]}],["Daghan ako isulti kanimo.\r",{"entities":[]}],["Nagreserba ra ko.\r",{"entities":[]}],["Kung magsugod ka sa imong kampanya, buhaton namon.\r",{"entities":[]}],["Kini ang imong nightmare, naghulga si Duterte nga tapuson ang iyang termino sa Hunyo 2022 .\r",{"entities":[[38,45,"YEAR"],[79,84,"MONTH"],[85,89,"YEAR"]]}],["Gipahiwatig pa ni Duterte si Robredo bahin sa kung diin ang bise presidente matag gabii.\r",{"entities":[[18,25,"PERSON"],[29,36,"PERSON"],[60,75,"PERSON"]]}],["Sa iyang Twitter, nag-post si Robredo usa ka video sa iyang kawani sa opisina nga nag-repack sa mga relief goods alang sa mga biktima sa bagyo\r",{"entities":[[30,37,"PERSON"],[126,133,"PERSON"]]}],["Kung ang usa ka Presidente usa ka misogynist, ang panagsulti moubus sa kini nga lebel.\r",{"entities":[[16,26,"PERSON"]]}],["Kini ang among gibuhat matag gabii, pagmata og pila ka semana aron, matag adlaw, makahatag ra kami og tabang sa mga nanginahanglan, ingon ni Robredo.\r",{"entities":[[141,148,"PERSON"]]}],["Sa iyang uban pang mga post, nagtoo ang bise presidente nga nahatagan na usab si Duterte og sayup nga kasayuran busa gibawog kini .\r",{"entities":[[45,55,"PERSON"],[81,88,"PERSON"]]}],["Ang kampo ni Robredo kaniadto gisaway ang punong abogado sa ligal nga pagkapresidente nga si Salvador Panelo sa pagpamakak sa diha nga giingon sa naulahi nga ang bise presidente migamit usa ka eroplano nga C-130 sa militar sa pag-adto sa Catanduanes\r",{"entities":[[13,20,"PERSON"],[49,56,"PERSON"],[93,108,"PERSON"],[162,177,"PERSON"],[238,249,"LOCATION"]]}],["Giinsistir ni Robredo, gituman ra niya ang iyang katungdanan aron matabangan ang mga tawo nga nanginahanglan panahon sa mga katalagman .\r",{"entities":[[14,21,"PERSON"]]}],["Daghan sa inyo ang nagtabang sa pagtubag sa mga naghilak ug nangayo tabang nga maluwas sa Cagayan ug Isabela sa gabii sa Nobyembre 13 hangtod sa aga sa Nobyembre 14.\r",{"entities":[[90,97,"LOCATION"],[101,108,"LOCATION"],[121,130,"MONTH"],[131,133,"DAY"],[152,161,"MONTH"],[162,164,"DAY"]]}],["1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 20010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 \r",{"entities":[[0,4,"YEAR"],[5,9,"YEAR"],[10,14,"YEAR"],[15,19,"YEAR"],[20,24,"YEAR"],[25,29,"YEAR"],[30,34,"YEAR"],[35,39,"YEAR"],[40,44,"YEAR"],[45,49,"YEAR"],[50,54,"YEAR"],[55,59,"YEAR"],[60,64,"YEAR"],[65,69,"YEAR"],[70,74,"YEAR"],[75,79,"YEAR"],[80,84,"YEAR"],[85,89,"YEAR"],[90,94,"YEAR"],[95,99,"YEAR"],[100,105,"YEAR"],[106,110,"YEAR"],[111,115,"YEAR"],[116,120,"YEAR"],[121,125,"YEAR"],[126,130,"YEAR"],[131,135,"YEAR"],[136,140,"YEAR"],[141,145,"YEAR"],[146,150,"YEAR"],[151,155,"YEAR"],[156,160,"YEAR"],[161,165,"YEAR"],[166,170,"YEAR"]]}],["\r",{"entities":[]}],["Niadtong 1992 gihagit Comelec Chairman Christian Monsod ang mga kandidato nga nidagan nga boot magpa ekzemp sa gun ban, og niingon \"kung di ninyo maagwanta, panggawas mo sa kusina sa botohan\".\r",{"entities":[[22,29,"ORG"],[30,55,"PERSON"]]}],["adtong Enero adtong Pebrero adtong Marso adtong Abril adtong Mayo adtong Hunyo adtong Hulyo adtong Agosto adtong Setyembre adtong Oktubre adtong Nobyembre adtong Disyembre\r",{"entities":[[7,12,"MONTH"],[20,27,"MONTH"],[35,40,"MONTH"],[48,53,"MONTH"],[61,65,"MONTH"],[73,78,"MONTH"],[86,91,"MONTH"],[99,105,"MONTH"],[113,122,"MONTH"],[130,137,"MONTH"],[145,154,"MONTH"],[162,171,"MONTH"]]}],["\r",{"entities":[]}],["Linggo Lunes Martes Miyerkules Huwebes Biyernes Sabado\r",{"entities":[[0,6,"DAY"],[7,12,"DAY"],[13,19,"DAY"],[20,30,"DAY"],[31,38,"DAY"],[39,47,"DAY"],[48,54,"DAY"]]}],["\r",{"entities":[]}],["linggo lunes martes miyerkules huwebes biyernes sabado\r",{"entities":[[0,6,"DAY"],[7,12,"DAY"],[13,19,"DAY"],[20,30,"DAY"],[31,38,"DAY"],[39,47,"DAY"],[48,54,"DAY"]]}],["Sumala pa ni Duque, dili siya uyon nga maningil pa alang sa mas paspas nga resulta apan kung himuon kini, dili kini molapas sa P5,000.\r",{"entities":[[13,18,"PERSON"]]}],["\r",{"entities":[]}],["Kung ang usa ka pasilidad madakup nga nakalapas sa mando, mag-atubang sila sa pagsuspinde ug magbayad sa multa alang sa ikatulong kalapasan, ang laboratoryo sa lisensya mahimo’g bawion.\r",{"entities":[]}],["\r",{"entities":[]}],["Pinauyon sa Department of Health kaniadtong Miyerkules, ang kinatibuk-ang ihap sa COVID-19 nga mga kaso sa Pilipinas miabot sa 422,915 pagkahuman sa pagrehistro sa 1,202 nga mga bag-ong kaso.\r",{"entities":[[12,32,"ORG"],[44,54,"DAY"],[107,116,"LOCATION"]]}],["\r",{"entities":[]}],["Nakadayeg ang aktor nga si Tony Labrusca sa kaayo ug pag-atiman sa aktres nga si Heaven Peralejo.\r",{"entities":[[14,19,"PERSON"],[27,40,"PERSON"],[67,73,"PERSON"],[81,96,"PERSON"]]}],["\r",{"entities":[]}],["Nagkauban sila Tony ug Heaven sa labing bag-ong serye sa ABS-CBN nga Bagong Umaga.\r",{"entities":[[15,19,"PERSON"],[23,29,"PERSON"],[57,64,"ORG"]]}],["\r",{"entities":[]}],["Sa Maayong Kinabuhi kaniadtong Martes, gidetalye ni Tony ang maayong mga kalidad sa Langit nga gihulagway pa niya nga iyang kaalyado.\r",{"entities":[[31,37,"DAY"],[52,56,"PERSON"]]}],["\r",{"entities":[]}],["Ginaapresyar ko lang si Heaven nga isa usab siya ka manggihatagon kaayo nga artista ug giatiman niya ako sa among mga eksena nga kami mga kaalyado.\r",{"entities":[[24,30,"PERSON"],[76,83,"PERSON"]]}],["\r",{"entities":[]}],["Tungod kay pareho pa gihapon kita nga bag-o sa industriya busa nag-atiman kami, ingon ni Tony.\r",{"entities":[[47,57,"ORG"],[89,93,"PERSON"]]}],["\r",{"entities":[]}],["Suma pa sa aktor nga nakadayeg usab siya sa gusto ni Heaven nga makilala ang tanan niya nga mga kauban sa trabaho.\r",{"entities":[[11,16,"PERSON"],[53,59,"PERSON"]]}],["\r",{"entities":[]}],["Mao nga kana ang akong naapresyar bahin kaniya tungod kay kaming tanan naka-lock in, ang kahago sa among gibuhat.\r",{"entities":[]}],["\r",{"entities":[]}],["Mao kana ang akong naapresyar bahin sa Langit, gusto niya nga kitang tanan okay, kanunay, ingon ni Tony.\r",{"entities":[[99,103,"PERSON"]]}],["\r",{"entities":[]}],["Sa gipangutana kung ang ilang gugma sa Langit matuman, mitubag si Tony: mahimo ba sila nga una nga managhigala.\r",{"entities":[[66,70,"PERSON"]]}],["\r",{"entities":[]}],["Tungod kay gipasabot nako nga nagkaila ra mi sa usag usa, unya solid kaayo ang among bonding busa okay lang nga mag-una gyud kaming tanan.\r",{"entities":[]}],["\r",{"entities":[]}],["Kahinumduman nga katapusang na-link ang Langit sa anak sa magtiayong Manny ug Jinkee Pacquiao Jimuel.\r",{"entities":[[69,74,"PERSON"],[78,84,"PERSON"],[85,93,"PERSON"],[94,100,"PERSON"]]}],["\r",{"entities":[]}],["Gawas nila Tony ug Heaven, ang mga bituon sa bag-ong serye sa ABS-CBN nga Bagong Umaga mao sila Kiko Estrada, Michelle Vito, Yves Flores, ug Barbie Imperial.\r",{"entities":[[11,15,"PERSON"],[19,25,"PERSON"],[62,69,"ORG"],[96,108,"PERSON"],[110,123,"PERSON"],[125,129,"PERSON"],[130,136,"PERSON"],[141,156,"PERSON"]]}],["\r",{"entities":[]}],["Mahimo nimong tan-awon ang Bagong Umaga gikan Lunes hangtod Biyernes, 2:30 sa hapon sa Kapamilya Channel sa cable ug satellite TV.\r",{"entities":[[46,51,"DAY"],[60,68,"DAY"],[87,104,"ORG"]]}],["\r",{"entities":[]}],["Anaa usab sa Kapamilya Online Live sa channel sa YouTube ug pahina sa Facebook sa ABS-CBN Entertainment, ingon man usab sa iwant app o iwanttfc.com nga gisalida usab ang serye sa A2Z Channel 11\r",{"entities":[[49,56,"ORG"],[70,78,"ORG"],[82,103,"ORG"]]}],["\r",{"entities":[]}],["Imbis nga magbakasyon kung gisuspinde ang mga klase sa online, nagtinabangay ang mga estudyante sa eskuylahan sa abogasya sa pagdonar alang sa mga nabiktima sa Bagyong Ulysses sa Cagayan Valley.\r",{"entities":[[85,95,"PERSON"],[99,109,"LOCATION"],[179,193,"LOCATION"]]}],["\r",{"entities":[]}],["Ang Tarlac State University School of Law (TSU-SOL) nagpahigayon usa ka akademikong pahulay sa usa ka semana gikan sa Nobyembre 17 hangtod 21.\r",{"entities":[[4,41,"ORG"],[42,50,"ORG"],[118,127,"MONTH"],[128,130,"DAY"],[139,141,"DAY"]]}],["\r",{"entities":[]}],["Kini tungod sa panawagan sa daghang mga estudyante nga magpahuway sa tunghaan tungod sa kakulang sa koneksyon sa internet ug pagkaguba sa kuryente nunot sa mga bagyo.\r",{"entities":[[40,50,"PERSON"]]}],["\r",{"entities":[]}],["Pinauyon kay (TSU-SOL) Dean Peng dela Rama, naengganyo nila ang mga estudyante sa balaod nga pahimuslan ang mga wala’y trabaho aron makaapil sa donasyon drive ug mag-empake sa mga relief goods sa gym sa unibersidad.\r",{"entities":[[13,22,"ORG"],[23,42,"PERSON"],[68,78,"PERSON"],[203,214,"ORG"]]}],["\r",{"entities":[]}],["Nahimo nila nga ma-repack ang hapit 2,000 nga sako nga bugas, tinapay, de-lata, tambal, mga banyo ug mineral water.\r",{"entities":[[36,41,"YEAR"]]}],["\r",{"entities":[]}],["Duha ka mga tawo ang nagpadala mga relief goods sa Sta. Maria ug Ilagan sa Isabela ug ingon man sa Tuguegarao City sa Cagayan.\r",{"entities":[[12,16,"PERSON"],[51,61,"LOCATION"],[65,71,"LOCATION"],[75,82,"LOCATION"],[99,114,"LOCATION"],[118,125,"LOCATION"]]}],["\r",{"entities":[]}],["Mga sundalo sa Philippine Army ug Philippine Air Force ang ningtabang sa paghatud sa kanila.\r",{"entities":[[4,11,"PERSON"],[15,30,"ORG"],[34,54,"ORG"]]}],["\r",{"entities":[]}],["Pinauyon kay dela Rama, hinungdanon ang temporaryong pagsuspenso sa klase alang sa mga estudyante tungod kay nakatabang kini sa mga nanginahanglan.\r",{"entities":[[13,22,"PERSON"],[87,97,"PERSON"]]}],["\r",{"entities":[]}],["Pagkahuman sa hapit 5 ka tuig, gidakup sa pulis ang usa ka senior citizen nga gipatay kuno ang iyang silingan sa Lungsod sa Quezon.\r",{"entities":[[20,29,"YEAR"],[42,47,"PERSON"],[124,130,"LOCATION"]]}],["\r",{"entities":[]}],["Ang suspetsado naila nga si Narciso Llanes, 65, nga nagtago-tago human gipusil ang iyang silingan kaniadtong Disyembre 2015.\r",{"entities":[[28,42,"PERSON"],[109,118,"MONTH"],[119,123,"YEAR"]]}],["\r",{"entities":[]}],["Nagsugod ang away sa usa ka yano nga away, adunay siya usa ka lalaki nga nagbabag sa usa ka motorsiklo sa dalan ug kini ang hinungdan kung giunsa gipatay ni Llanes ang silingan.\r",{"entities":[[62,68,"PERSON"],[92,102,"PERSON"],[157,163,"PERSON"],[168,176,"PERSON"]]}],["\r",{"entities":[]}],["Ang suspek gisirkunsidohan ang mga probinsya sa amihanan sa pagtago hangtod nga nadakup siya sa pulisya sa Maragondon, Cavite kaniadtong Martes, nabilanggo na siya karon ug nag-atubang sa kasong pagpatay.\r",{"entities":[[4,10,"PERSON"],[119,125,"LOCATION"],[126,143,"DAY"]]}],["\r",{"entities":[]}],["Ang syudad sa Valenzuela maghatag P1.2 milyon aron maghatag kasayuran kung asa ang mga suspetsado sa pagpatay sa usa ka rider kaniadtong Oktubre.\r",{"entities":[[11,24,"LOCATION"],[120,125,"PERSON"],[126,144,"MONTH"]]}],["\r",{"entities":[]}],["Upat pa nga mga suspek ang gipangita karon, lakip na ang usa ka pulis kaniadto, si Patrolman Anthony Cubos, nga giingon nga lider sa usa ka criminal gang.\r",{"entities":[[16,22,"PERSON"],[83,106,"PERSON"],[124,129,"PERSON"]]}],["Si Anacleta Alconera gusto lang nga magpabilin sa balay sa residente apan giingon niya nga kinahanglan siya mogawas aron makapangita trabaho .\r",{"entities":[[3,20,"PERSON"],[59,68,"PERSON"]]}],["\r",{"entities":[]}],["Giasoy ni Verceles ang hitabo sa PNP Regional Police Office 1 sa La Union, apan wala gihapon makuha ang account.\r",{"entities":[[10,18,"PERSON"],[33,59,"ORG"],[65,73,"LOCATION"]]}],["\r",{"entities":[]}],["Pinauyon kay Lady Ann Salem, ang tigbalita nga usa sa mga naaresto kaniadtong Disyembre 10, giingon nga gibalibaran sila sa pulisya sulod sa usa ka oras ug nagtanum og ebidensya, sa wala pa gihimo ang pagpangita.\r",{"entities":[[13,27,"PERSON"],[78,87,"MONTH"],[88,90,"DAY"]]}],["\r",{"entities":[]}],["Sa Philippine General Hospital, pananglitan, sa pagkakaron adunay 97 nga aktibo nga mga kaso, mas gamay kumpara sa miaging 200 nga mga kaso.\r",{"entities":[[3,30,"ORG"]]}],["\r",{"entities":[]}],["Giawhag sa senador ang Senate Committee on National Defense nga madungog dayon ang Senate Resolution 137 nga iyang gisumite kaniadtong 2019 nga naghangyo nga usisaon ang kasabutan sa AFP-Dito.\r",{"entities":[[23,59,"ORG"],[135,139,"YEAR"],[183,191,"ORG"]]}],["\r",{"entities":[]}],["Si Las Piñas City Rep. Camille Villar ginomina pud sa representate na mamumulong apan nagdumili kini.\r",{"entities":[[3,17,"LOCATION"],[18,37,"PERSON"]]}],["\r",{"entities":[]}],["Una na nga giasabot ni UP Political Science Assistant Professor Jean Franco nga samtang magdaghan ang mga representante na mamumulong madungagan pud ang gastos sa Kamara.\r",{"entities":[[23,43,"ORG"],[44,75,"PERSON"],[163,169,"ORG"]]}],["\r",{"entities":[]}],["Mubalik pag-ablli ang klase sa Enero 4, sulti sa DepEd sa ilang facebook page.\r",{"entities":[[31,36,"MONTH"],[37,38,"DAY"],[49,54,"ORG"],[64,72,"ORG"]]}],["\r",{"entities":[]}],["Nagpahibalo diri. Lockdown karong Disyembre 22- Enero 2, sulti sa usa ka tweet.\r",{"entities":[[34,43,"MONTH"],[44,47,"DAY"],[48,53,"MONTH"],[54,55,"DAY"]]}],["\r",{"entities":[]}],["Gipailawomi ni presidente Rodrigo Duterte ang Metro Manila, Batangas, Ilo-ilo City, Tacloban City, Lanao del Sur, Iligan ug Davao City sa general quarantine hangtod sa katapusan sa Disyembre.\r",{"entities":[[15,25,"PERSON"],[26,41,"PERSON"],[46,58,"LOCATION"],[60,68,"LOCATION"],[70,82,"LOCATION"],[84,97,"LOCATION"],[99,112,"LOCATION"],[114,120,"LOCATION"],[124,134,"LOCATION"],[181,190,"MONTH"]]}],["\r",{"entities":[]}],["Nahimong init nga isyu ang red-tagging pagkahuman nagpasidaan si AFP Habagatang Luzon Command Chief Lt. Gen. Antonio Parlade Jr. sa ila Liza Soberano, Angel Locsin, ug Katriona Gray nga dili magpagamit sa mga kalabang grupo na front sa CPP.\r",{"entities":[[65,68,"ORG"],[86,128,"PERSON"],[136,149,"PERSON"],[151,163,"PERSON"],[168,181,"PERSON"],[236,239,"ORG"]]}],["\r",{"entities":[]}],["Adtong 2018, nakadawat siya og award para sa iyang gihulma nga takore nga iyang gitawag nga Vigina teapot.\r",{"entities":[[7,11,"YEAR"]]}],["\r",{"entities":[]}],["Nadakpan atong Huwebes sa Cebu City ang wala nailhang lalaki nga nagpakatag sa messaging app ug bastos nga litrato sa usa ka 19 na anyos nga babaye sulti sa Regional Anti-Cybercrime Unit sa Central Visayas (RACU-7).\r",{"entities":[[15,22,"DAY"],[26,35,"LOCATION"],[157,186,"ORG"],[190,205,"LOCATION"]]}],["\r",{"entities":[]}],["Adtong Oktubre 16, nagpadala na ang gobyerno sa syudad sa CDI og sulat para hunongon ang iligal nga mga aktibidad sa creek apan wala nisunod ang kumpanya.\r",{"entities":[[7,14,"MONTH"],[15,17,"DAY"],[58,61,"ORG"]]}],["\r",{"entities":[]}],["Sa nahimong panudlo sa Department of Health (DOH) ug mga ahensya nga nagdumala sa COVID19 pagbakuna sa House Committee on People's Participation gibungat niini ang mga sektor nga prioridad nga makakuha sa bakuna.\r",{"entities":[[23,49,"ORG"],[103,144,"ORG"]]}],["\r",{"entities":[]}],["Sumala kay ERC Chiarperson ug CEO Agnes Devanadera, nagdala og kagubot ug pagkalibog gikan sa mga customer nga wala naghatag ug tama nga impormasyon sa Meralco samtang naa tunga sa pandemya ang nasud.\r",{"entities":[[11,26,"PERSON"],[30,50,"PERSON"],[152,159,"ORG"]]}],["\r",{"entities":[]}],["Gidasig ni Senador Nancy Binay ang Philippine Overseas Employment Administration aug ang Inter Agency Task Force nga bawion na ang deployment ban sa healthcare workers.\r",{"entities":[[11,30,"PERSON"],[35,80,"ORG"],[89,112,"ORG"],[149,167,"PERSON"]]}],["\r",{"entities":[]}],["Paklaro ni Binay wala'y karapatan ang gobyerno nga pugngan ang mga health worker kung gusto nila magtrabaho sa ubang nasud para buhion ang ilahang pamilya.\r",{"entities":[[11,16,"PERSON"],[38,46,"PERSON"],[147,154,"PERSON"]]}],["\r",{"entities":[]}],["Sulti niya kung magmagahi usab ang POEA sa deployment ban, dapat siguruon sa gobyerno nga adunay tama na kompensasyon ang mahatag sa mga niini.\r",{"entities":[[35,39,"ORG"],[77,85,"ORG"]]}],["\r",{"entities":[]}],["Sa nakapanayam ni Tulfo si Undersecretary Ricogyudge Echiverri sa department of the Interior and Local Government (DILG) may kalabutan sa isyu.\r",{"entities":[[18,23,"PERSON"],[27,62,"PERSON"],[66,120,"ORG"]]}],["",{"entities":[]}],["",{"entities":[]}]]}

spaCy comes with pre-built models for lots of languages by I customized it using 
".blank" instead of ".load".

then create an empty model, give it a name, then add a simple pipeline to it.

spaCy you can do much more than just entity extraction

In [ ]:
nlp = spacy.blank('en')
nlp.vocab.vectors.name = "demo"
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last = True)

ValueError: ignored

Before training, we need to make our model aware of the possible entities. To do that, we add all the labels we’re aware of:

In [ ]:
for label in training_data["classes"]:
  print(label)
  nlp.entity.add_label(label)


 We’ll need to allocate the models and get an optimizer via our model’s begin_training() method

In [ ]:
optimizer = nlp.begin_training()

Then we update the model with our training data. Each text, with its annotations (those labeled entities), would be passed to the update() function of our model.

we have little training data, we just loop through it a few times, in random order

In [ ]:
for iter in range(5):
  losses={}
  
  for text, annotations in training_data["annotations"]:
      if len(text) > 0:
        print([text])
        print(annotations)
        nlp.update([text], [annotations], sgd=optimizer, losses = losses)
        print(losses)
        

In [ ]:
optimizer = nlp.begin_training()

**The model we just built is already loaded in memory. If you don’t want to train it every time, you can use provided list of data and load it when needed.**

In [ ]:
test = pd.read_excel('Cebuano.Parallel.Wikipedia.xlsx', names=["Cebuano"])

**With the model loaded, you’ll use it to predict entities just as you would with a pre-built model:**

In [ ]:
num = 0
text=[]
label=[]
for i in range(len(test)):
  doc=nlp(test.iloc[num][0])
  print(" - ", doc ," - ")
  for ent in doc.ents:
    print(ent.text, ent.label_)
    text.append(ent.text)

  num = num + 1  
  print("===================")  
  

 -  Kini ra ang istruktura sa intramuros kaniadtong nabuhi ang pagluwas sa Manila kaniadtong 1945.  - 
Manila ORG
 -  Si Miag-ao nahimong usa ka independente nga parokya kaniadtong 1731, kaniadtong gitukod ang usa ka yaon nga simbahan ug kombento.  - 
Miag- LOCATION
independente PERSON
 -  Bisan pa, ang pagkaguba sa lungsod sa mga pirata nga Muslim niadtong 1741 ug 1754 nagdala sa gitukod pag-usab nga lungsod sa mas luwas nga lokasyon.  - 
1741 DAY
1754 YEAR
 -  Ang bag-ong simbahan, nga gitukod niadtong 1787-97, gitukod ingon usa ka kuta, aron maatubang ang dugang nga pag-atake.  - 
1787 YEAR
- DAY
 -  Bisan pa, grabe ang nadaot sa panahon sa rebolusyon batok sa Espanya kaniadtong 1898 ug sa panahon sa ikaduhang gubat sa kalibutan.  - 
Espanya ORG
1898 DAY
 -  Duha ka mga torre sa kampanilya ang gidugang kaniadtong 1854, apan ang hilera nga nakasulod sa linog sa 1880 ug kinahanglan nga buwagon.  - 
torre PERSON
1854 DAY
1880 ug GPE
 -  Tungod sa katalagman sa mga natural nga kalamidad

KeyboardInterrupt: ignored

In [ ]:
#doc = nlp("Gipimbestigahan ni Finance Secretary Carlos Dominguez III sa Bureau of Internal Revenue (BIR) ug sa Bureau of Customs tungod sa giingon nga paggamit sa mga pribadong negosyante sa mga kooperatiba isip dummies sa pag-import og bugas humay.")
doc = nlp("Bisan kung gusto sa Pangulo nga pondohan ang usa ka ahensya sa gobyerno o pondohan ang usa ka proyekto, wala kini gahum nga maghatag isyu salapi sa National Treasury tungod kay kini nga gahum talagsaon sa Kongreso.")
#doc = nlp("Kung nahitabo kini, adunay posibilidad nga ang Kongreso sa Estados Unidos dili madeklara nga presidente ug bise-presidente sa wala pa ang hapon sa Enero 20, 2021.")
#doc = nlp("Gikan ngadto sa Nobyembre 05, ang mga pagpangubkob sa arkeolohiya, nga gipanguluhan ni Alfredo Evangelista, nahimo sa dapit sa rock shelter.")
#doc = nlp("Sa Tsina nagpanuko kami sa pag - ila nga ang mga sumbanan sa sinultihan gapos sa sosyal nga klase. Ang una nga mga makina nga nagpatik nag - abut sa Estados Unidos kaniadtong tungatunga sa 1600, nga nagtimaan sa usa ka hinungdanon nga lakang sa kasaysayan sa libre nga press sa Amerika.")

In [ ]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Pangulo PERSON
gobyerno ORG
National Treasury ORG
Kongreso ORG


In [ ]:
W = []
uniquetoken = 0
words = 0
for text, annotations in training_data["annotations"]:
  token = text.split()
  a_set = set(token)
  number_of_unique_token = len(a_set)
  uniquetoken = number_of_unique_token + uniquetoken
  ans1= len(token)
  words = words + ans1
  W.append(ans1)

print("Number of sentence in all annotation =",len(W)) 
print("Number of token in all annotation =",words)
print("Number of unique token in all annotation =",uniquetoken) 
average = Average(W)
print("Average words per sentence =", round(average, 2)) 

Number of sentence in all annotation = 206
Number of token in all annotation = 3378
Number of unique token in all annotation = 2869
Average words per sentence = 16.4


In [ ]:
annot=[]
PERSON=0
ORG=0
LOC=0
MONTH = 0
DAY=0
YEAR = 0
GPE = 0
num= 0

for text, annotations in training_data["annotations"]:
  annon = str(annotations)
  annot.append(annon)
  #tst = annot[0].count("PERSON")
  p = annot[num].count("PERSON")
  PERSON = PERSON + p 
  org = annot[num].count("ORG")
  ORG = ORG + org 
  loc = annot[num].count("LOCATION")
  LOC = LOC + loc
  mnt = annot[num].count("MONTH")
  MONTH= MONTH + mnt
  dy = annot[num].count("DAY")
  DAY= DAY + dy
  yr = annot[num].count("YEAR")
  YEAR= YEAR + yr
  gpe = annot[num].count("GPE")
  GPE= GPE + gpe
  num = num + 1
  
print("Total number of Entity |PERSON| in annotation  =",PERSON)
print("Total number of Entity |ORGANIZATION| in annotation  =",ORG)
print("Total number of Entity |LOCATION| in annotation  =",LOC)
print("Total number of Entity |MONTH| in annotation  =",MONTH)
print("Total number of Entity |DAY| in annotation  =",DAY)
print("Total number of Entity |YEAR| in annotation  =",YEAR)
print("Total number of Entity |COUNTRY| in annotation  =",GPE)



Total number of Entity |PERSON| in annotation  = 171
Total number of Entity |ORGANIZATION| in annotation  = 94
Total number of Entity |LOCATION| in annotation  = 44
Total number of Entity |MONTH| in annotation  = 28
Total number of Entity |DAY| in annotation  = 39
Total number of Entity |YEAR| in annotation  = 46
Total number of Entity |COUNTRY| in annotation  = 22
